<a href="https://colab.research.google.com/github/Gratisfo/Parentents-and-children/blob/main/topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [5]:
data = pd.read_csv('data.csv')
del data['Unnamed: 0']
data = data.dropna().reset_index(drop=True)
data

,speaker,text,role,date,title
0,#taratora,"И ты еще, мошенник, смеешь мне указывать! Да з...",parent,1788,krylov-prokazniki
1,#taratora,То есть чего-нибудь хорошенького.,parent,1788,krylov-prokazniki
2,#taratora,"Как, бездельник! да разве не для того он сюда ...",parent,1788,krylov-prokazniki
3,#taratora,"Да его ль вина, бестия, что вы все здесь ходит...",parent,1788,krylov-prokazniki
4,#taratora,Я окончала!.. Как я рада! Элегия эта беспример...,parent,1788,krylov-prokazniki
...,...,...,...,...,...
20297,#sineus,"Ты, сказывают, хотел Славян и прочие народы от...",children,1786,ekaterina-vtoraja-iz-zhizni-rjurika
20298,#truvor,Какие же ты имел при том намерения?,children,1786,ekaterina-vtoraja-iz-zhizni-rjurika
20299,#oskold,"К чему прение тут, где дело само по себе ясно?",children,1786,ekaterina-vtoraja-iz-zhizni-rjurika
20300,#rjurik,"Бодрость духа твоего, князь Вадим, не унывает;...",parent,1786,ekaterina-vtoraja-iz-zhizni-rjurika


In [7]:
def prop_date(date):
  try:
    return int(date)
  except:
    return int(date[:4]) 

data['date'] = data['date'].apply(lambda x: prop_date(x))

# Preprocessing

In [ ]:
!pip install razdel

In [ ]:
!pip install pymorphy2[fast]

In [ ]:
import re
import pymorphy2
import nltk
from razdel import sentenize, tokenize
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()

In [54]:
def preproc_text(text):
  # tokenize
  tokens = [_.text for _ in list(tokenize(text))]
  
  # lower register
  tokens = [w.lower() for w in tokens]
  
  # remove puntuation
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  
  # remove numbers, save only letters
  words = [word for word in stripped if word.isalpha()]
  
  # delete stop-words
  stop_words = stopwords.words('russian')
  stop_words.extend(['мой', 'твой', 'ваш', 'свой', 'такой', 'какой', 'наш', 'который'])
  stop_words = set(stop_words)
  words = [w for w in words if not w in stop_words]

  # to normal form
  allowed = ['NOUN', 'ADJF', 'ADJS', 'VERB', 'ADVB']
  lemmas = [morph.parse(w)[0].normal_form for w in words if morph.parse(w)[0].tag.POS in allowed] 
  
  return lemmas

In [55]:
data['clean_text'] = data['text'].apply(lambda x: preproc_text(x)) 

In [56]:
# Join replicas to text for each speaker
join_text = data.groupby(['speaker']).agg(list)
join_text['role'] = join_text['role'].apply(lambda x: x[0])
join_text['date'] = join_text['date'].apply(lambda x: x[0])
join_text['title'] = join_text['title'].apply(lambda x: x[0])
join_text

,text,role,date,title,clean_text
speaker,,,,,
#Anja,"[Пройдемте здесь. Ты, мама, помнишь, какая это...",children,1903,chekhov-vishnevyi-sad,"[[пройти, мама, помнить, комната], [спасть, до..."
#Dashenka,[Они хочут свою образованность показать и всег...,children,1889,chekhov-svadba,"[[хотеть, образованность, говорить, непонятный..."
#FedorIvanovich,"[Здорово, ребята!, Жарко! Водки выпить надо., ...",children,1889,chekhov-leshii,"[[ребята], [жарко, водка], [юля], [бросить, юл..."
#GordejKarpych,"[Что распелись! Горланят, точно мужичье!, А эт...",parent,1853,ostrovsky-bednost-ne-porok,"[[распеться, горланить, точно, мужичий], [глуп..."
#Larisa,"[До свиданья, Вася!, Я сейчас все за Волгу смо...",children,1874,ostrovsky-bespridannitsa,"[[свидание, вася], [волга, смотреть, тот, стор..."
...,...,...,...,...,...
#vtoraja_knjazhna,"[Какие складочки!, Про это знает целый свет.]",children,1822,griboyedov-gore-ot-uma,"[[какой, складочка], [знать, целый, свет]]"
#zamir,"[Приехал мой отец! Какой мне день счастливый!,...",children,1784,knyazhnin-hvastun,"[[приехать, отец, день, счастливый], [клятвопр..."
#zasypkin,"[А уж как я рад, что вы приехали к нам, Анисья...",parent,1886,mamin-sibirjak-zolotopromyshlenniki,"[[рад, приехать, анисья, тихонович, рад, ангел..."


In [57]:
# split texts by period
data18 = join_text[(join_text['date'] > 1700) & (join_text['date'] < 1799)]
data19 = join_text[(join_text['date'] > 1800) & (join_text['date'] < 1899)]
data20 = join_text[join_text['date'] > 1900]

In [58]:
# split texts by role
parents_18 = data18[data18['role']=='parent'].clean_text.values
parents_18 =  [item for sublist in parents_18 for item in sublist]

children_18 = data18[data18['role']=='children'].clean_text.values
children_18 =  [item for sublist in children_18 for item in sublist]

parents_19 = data19[data19['role']=='parent'].clean_text.values
parents_19 =  [item for sublist in parents_19 for item in sublist]

children_19 = data19[data19['role']=='children'].clean_text.values
children_19 =  [item for sublist in children_19 for item in sublist]

parents_20 = data20[data20['role']=='parent'].clean_text.values
parents_20 =  [item for sublist in parents_20 for item in sublist]

children_20 = data20[data20['role']=='children'].clean_text.values
children_20 =  [item for sublist in children_20 for item in sublist]

# Topic Modeling

In [ ]:
!pip install pyLDAvis

In [ ]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models 
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [24]:
def builg_ngrams(data_words):
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)
  return bigram_mod, trigram_mod

def make_bigrams(texts):
    bigram_mod, trigram_mod = builg_ngrams(texts)
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    bigram_mod, trigram_mod = builg_ngrams(texts)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def topic_modeling(words):
    bigram, trigram = builg_ngrams(words)
    id2word = corpora.Dictionary(words)
    corpus = [id2word.doc2bow(text) for text in words]
    return corpus, id2word

#LDA model for parents' texts

## 18 centure

In [ ]:
corpus, id2word = topic_modeling(parents_18)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [82]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.083*"свой" + 0.049*"быть" + 0.047*"человек" + 0.035*"пойти" + 0.023*"дом" '
  '+ 0.017*"виноватый" + 0.015*"всякий" + 0.014*"надежда" + 0.014*"год" + '
  '0.014*"вон"'),
 (1,
  '0.148*"мой" + 0.072*"твой" + 0.028*"думать" + 0.027*"сударыня" + '
  '0.027*"сделать" + 0.025*"должный" + 0.020*"помиловать" + 0.014*"воля" + '
  '0.013*"жена" + 0.012*"злодей"'),
 (2,
  '0.109*"сударь" + 0.059*"мочь" + 0.055*"дело" + 0.053*"весь" + 0.030*"очень" '
  '+ 0.022*"радость" + 0.017*"изволить" + 0.015*"деньга" + 0.015*"ум" + '
  '0.013*"вид"'),
 (3,
  '0.039*"сказать" + 0.033*"взять" + 0.032*"сам" + 0.031*"стать" + '
  '0.030*"время" + 0.026*"иметь" + 0.023*"бояться" + 0.022*"казаться" + '
  '0.020*"сердце" + 0.020*"жизнь"'),
 (4,
  '0.049*"ваш" + 0.040*"право" + 0.030*"дать" + 0.030*"любить" + 0.027*"друг" '
  '+ 0.020*"оставить" + 0.016*"жена" + 0.015*"ведать" + 0.012*"будить" + '
  '0.012*"слушаться"'),
 (5,
  '0.063*"наш" + 0.045*"муж" + 0.043*"сей" + 0.036*"государь" + 0.029*"тот" + '


In [ ]:
# Compute Perplexity
Perplexity = lda_model.log_perplexity(corpus)   # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=parents_18, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [84]:
print('\nPerplexity: ', Perplexity )
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.796771522685829

Coherence Score:  0.5528117558727152


In [85]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.313151  0.072748       1        1  12.196025
0      0.107465  0.246067       2        1  11.159875
8     -0.060323 -0.105948       3        1  11.142290
2      0.044789 -0.214544       4        1  10.532950
6      0.039219 -0.007477       5        1   9.638426
9      0.027068  0.021359       6        1   9.386125
7      0.102281 -0.015882       7        1   9.189943
5      0.037655  0.008695       8        1   9.135400
3      0.041706  0.006340       9        1   9.076992
4     -0.026711 -0.011359      10        1   8.541973, topic_info=          Term        Freq       Total Category  logprob  loglift
97         мой  237.000000  237.000000  Default  30.0000  30.0000
1437    сударь  151.000000  151.000000  Default  29.0000  29.0000
9         свой  121.000000  121.000000  Default  28.0000  28.0000
31        твой  116.000000  116.000000  Default  27.0000  27.0000
821       душа   91.000000   91.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2332   простой    5.819781    6.481610  Topic10  -5.2597   2.3525
790      замуж    5.718771    6.383511  Topic10  -5.2772   2.3502
1361  почтение    5.669929    6.332002  Topic10  -5.2858   2.3497
1         жена   17.941657   56.667547  Topic10  -4.1338   1.3101
82    говорить    9.580518   50.019721  Topic10  -4.7612   0.8075

[338 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3858      9  0.919502     батожьё
408       3  0.992307     батюшка
774       3  0.986739        беда
656       8  0.951168  бездельник
290       6  0.985102         бог
...     ...       ...         ...
139       1  0.963541       честь
1200      1  0.962847       чудно
3860      2  0.986713       шайка
324       3  0.944169          эк
224       4  0.943122        этот

[308 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 9, 3, 7, 10, 8, 6, 4, 5])

## 19 centure

In [ ]:
corpus, id2word = topic_modeling(parents_19)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [88]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.051*"один" + 0.022*"брать" + 0.021*"чепрак" + 0.020*"кондратьич" + '
  '0.020*"тихон" + 0.017*"домой" + 0.013*"беда" + 0.013*"выйти" + '
  '0.011*"простить" + 0.011*"дама"'),
 (1,
  '0.070*"сказать" + 0.055*"мой" + 0.046*"видеть" + 0.035*"любить" + '
  '0.028*"рука" + 0.020*"точно" + 0.016*"делать" + 0.013*"грех" + 0.011*"саша" '
  '+ 0.011*"сын"'),
 (2,
  '0.103*"дело" + 0.046*"быть" + 0.029*"другой" + 0.024*"голова" + '
  '0.018*"этот" + 0.017*"муж" + 0.017*"пора" + 0.015*"сюда" + 0.014*"матушка" '
  '+ 0.013*"самый"'),
 (3,
  '0.042*"ваш" + 0.023*"сделать" + 0.023*"понимать" + 0.018*"стоить" + '
  '0.018*"господин" + 0.016*"белоносов" + 0.016*"молчать" + 0.016*"народ" + '
  '0.014*"разговор" + 0.013*"послать"'),
 (4,
  '0.063*"говорить" + 0.060*"человек" + 0.041*"знать" + 0.033*"такой" + '
  '0.032*"хотеть" + 0.028*"деньга" + 0.023*"бог" + 0.022*"твой" + '
  '0.020*"очень" + 0.017*"хороший"'),
 (5,
  '0.063*"свой" + 0.040*"мочь" + 0.036*"дом" + 0.031*"казаться" + '
  '0.02

In [ ]:
# Compute Perplexity
Perplexity = lda_model.log_perplexity(corpus)   # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=parents_19, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [90]:
print('\nPerplexity: ', Perplexity )
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.566495988882286

Coherence Score:  0.4561042058041763


In [91]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.366503 -0.000888       1        1  19.869574
1     -0.041245 -0.039993       2        1   9.659247
5     -0.037445 -0.034708       3        1   9.609953
3     -0.042105 -0.075890       4        1   9.476760
2     -0.039875  0.299519       5        1   9.435942
7     -0.039947 -0.033834       6        1   8.992034
6     -0.042337 -0.064720       7        1   8.933642
8     -0.042746 -0.013891       8        1   8.491227
9     -0.041193 -0.018383       9        1   8.240139
0     -0.039610 -0.017212      10        1   7.291482, topic_info=           Term        Freq       Total Category  logprob  loglift
91         дело  288.000000  288.000000  Default  30.0000  30.0000
116    говорить  368.000000  368.000000  Default  29.0000  29.0000
127     человек  356.000000  356.000000  Default  28.0000  28.0000
160     сказать  199.000000  199.000000  Default  27.0000  27.0000
41         свой  179.000000  179.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
388     слушать   13.279489   13.964674  Topic10  -5.0905   2.5682
1311    ожидать   13.037708   13.722901  Topic10  -5.1088   2.5672
465    внимание   13.007177   13.692433  Topic10  -5.1112   2.5671
2249       баба   11.831823   12.516950  Topic10  -5.2059   2.5622
1513  сказывать   11.672387   12.358123  Topic10  -5.2195   2.5614

[335 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3544      7  0.974374  алексей
827       8  0.982202   андрей
5002      8  0.996907   анисья
1581      8  0.971788  аннушка
2249     10  0.958700     баба
...     ...       ...      ...
1034      6  0.978304      шея
328       2  0.978595    шутка
369      10  0.922761     этак
141       5  0.986264   этакий
301       5  0.988455     этот

[314 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 6, 4, 3, 8, 7, 9, 10, 1])

## 20 centure

In [92]:
corpus, id2word = topic_modeling(parents_20)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [94]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.080*"идти" + 0.017*"сюда" + 0.017*"дядя" + 0.017*"обидно" + '
  '0.015*"николай" + 0.013*"михайло" + 0.012*"день" + 0.012*"работать" + '
  '0.011*"скоро" + 0.010*"трудиться"'),
 (1,
  '0.040*"человек" + 0.038*"сказать" + 0.029*"мочь" + 0.027*"мой" + '
  '0.026*"семён" + 0.022*"свой" + 0.020*"жена" + 0.020*"отец" + 0.019*"жизнь" '
  '+ 0.018*"ступать"'),
 (2,
  '0.088*"говорить" + 0.028*"быть" + 0.022*"великий" + 0.020*"слышать" + '
  '0.019*"весь" + 0.015*"ходить" + 0.014*"пора" + 0.012*"положить" + '
  '0.012*"друг" + 0.010*"поехать"'),
 (3,
  '0.062*"дело" + 0.038*"павел" + 0.036*"бог" + 0.026*"сын" + 0.024*"послать" '
  '+ 0.017*"должный" + 0.017*"сегодня" + 0.016*"твой" + 0.016*"просить" + '
  '0.013*"врать"'),
 (4,
  '0.028*"сам" + 0.027*"пойти" + 0.021*"говорить" + 0.020*"душа" + '
  '0.017*"слушать" + 0.015*"жить" + 0.015*"колюшка" + 0.015*"чувствовать" + '
  '0.014*"сердце" + 0.012*"остаться"'),
 (5,
  '0.071*"хотеть" + 0.035*"помнить" + 0.022*"понять" + 0.019*"сидеть

In [ ]:
# Compute Perplexity
Perplexity = lda_model.log_perplexity(corpus)   # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=parents_20, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [96]:
print('\nPerplexity: ', Perplexity )
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.13281553363572

Coherence Score:  0.5970926161392855


In [97]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.260276  0.116262       1        1  15.290035
9     -0.180055  0.181176       2        1  12.787710
6      0.007675  0.002197       3        1   9.546022
8     -0.005527 -0.193896       4        1   9.235766
7     -0.031107 -0.088938       5        1   9.234348
3     -0.028924  0.022973       6        1   9.188615
2      0.011303 -0.045724       7        1   9.179092
4      0.007853 -0.015959       8        1   9.138768
0     -0.011731  0.025784       9        1   8.510872
5     -0.029763 -0.003876      10        1   7.888771, topic_info=          Term       Freq      Total Category  logprob  loglift
115   говорить  47.000000  47.000000  Default  30.0000  30.0000
181      знать  37.000000  37.000000  Default  29.0000  29.0000
67        идти  32.000000  32.000000  Default  28.0000  28.0000
218     хотеть  26.000000  26.000000  Default  27.0000  27.0000
50        дело  27.000000  27.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
1622  ночевать   2.205690   2.874453  Topic10  -5.1256   2.2749
1336    виктор   2.017198   2.685032  Topic10  -5.2149   2.2537
995     дерево   1.983031   2.650079  Topic10  -5.2320   2.2498
989     старый   2.176892   5.227207  Topic10  -5.1388   1.6638
1648    прохор   2.206739   9.801640  Topic10  -5.1251   1.0487

[363 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1850      3  0.769099         агаа
1853      7  0.874704           ад
1591      4  0.873389      адвокат
346       2  0.952803    александр
828       4  0.962330         анна
...     ...       ...          ...
33        8  0.797904    чтонибыть
1168      8  0.852333  чувствовать
1912      3  0.769108         чудо
1896     10  0.860823       шипеть
210       3  0.822325         этот

[350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 7, 9, 8, 4, 3, 5, 1, 6])

#LDA model for children' texts

## 18 centure

In [ ]:
corpus, id2word = topic_modeling(children_18)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [102]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.032*"казаться" + 0.027*"мой" + 0.019*"мера" + 0.019*"глаз" + '
  '0.017*"судьба" + 0.014*"воля" + 0.014*"виноватый" + 0.013*"надежда" + '
  '0.011*"россия" + 0.011*"хахахах"'),
 (1,
  '0.114*"мой" + 0.042*"ваш" + 0.027*"идти" + 0.025*"сердце" + 0.018*"француз" '
  '+ 0.017*"право" + 0.016*"жизнь" + 0.015*"скоро" + 0.014*"дама" + '
  '0.014*"слышать"'),
 (2,
  '0.050*"отец" + 0.026*"изволить" + 0.023*"смерть" + 0.018*"взять" + '
  '0.017*"любезный" + 0.015*"таков" + 0.014*"напрасно" + 0.014*"ныне" + '
  '0.013*"имя" + 0.013*"сударь"'),
 (3,
  '0.064*"мочь" + 0.045*"любовь" + 0.042*"дело" + 0.032*"думать" + '
  '0.026*"постоять" + 0.024*"другой" + 0.018*"родитель" + 0.016*"ум" + '
  '0.015*"всякий" + 0.013*"мать"'),
 (4,
  '0.056*"человек" + 0.049*"который" + 0.049*"такой" + 0.029*"сказать" + '
  '0.028*"любить" + 0.023*"свет" + 0.021*"говорить" + 0.016*"надобный" + '
  '0.016*"крест" + 0.014*"жена"'),
 (5,
  '0.049*"сей" + 0.030*"русский" + 0.028*"очень" + 0.023*"добродетель" 

In [ ]:
# Compute Perplexity
Perplexity = lda_model.log_perplexity(corpus)   # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=children_18, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [118]:
print('\nPerplexity: ', Perplexity )
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.325303256915332

Coherence Score:  0.5599544837551791


In [105]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.317299  0.003195       1        1  14.671901
7     -0.016496  0.010415       2        1  11.362756
6      0.083574  0.220429       3        1  11.241524
2      0.025516  0.037542       4        1  10.961104
1      0.073300 -0.222602       5        1   9.990784
3      0.027524 -0.006997       6        1   8.800212
8      0.029705  0.003581       7        1   8.723580
0      0.035807 -0.076844       8        1   8.577020
5      0.033595  0.021005       9        1   8.211519
9      0.024776  0.010275      10        1   7.459600, topic_info=          Term       Freq      Total Category  logprob  loglift
12         мой  64.000000  64.000000  Default  30.0000  30.0000
218    матушка  42.000000  42.000000  Default  29.0000  29.0000
216    батюшка  40.000000  40.000000  Default  28.0000  28.0000
129     хотеть  30.000000  30.000000  Default  27.0000  27.0000
127    человек  38.000000  38.000000  Default  26.0000  26.0000
...        ...        ...        ...      ...      ...      ...
1871    подвиг   1.751991   2.377884  Topic10  -5.2848   2.2902
1882   считать   1.751989   2.377884  Topic10  -5.2848   2.2902
1884     виски   1.750741   2.377991  Topic10  -5.2855   2.2895
884   перемена   1.690396   2.316401  Topic10  -5.3205   2.2806
7      принять   1.459928   2.086691  Topic10  -5.4671   2.2385

[345 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
216       3  0.987205       батюшка
1009     10  0.787420          беда
1480      5  0.770314  благополучие
237       4  0.811466   благородный
140       7  0.969457           бог
...     ...       ...           ...
1450      1  0.777244      чужехват
1063      3  0.796524         чужой
248       6  0.794832         шутка
303       9  0.953390          этот
502       9  0.740212          ясно

[321 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 8, 7, 3, 2, 4, 9, 1, 6, 10])

## 19 centure

In [ ]:
corpus, id2word = topic_modeling(children_19)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [108]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.037*"делать" + 0.032*"время" + 0.027*"дать" + 0.022*"тяжело" + '
  '0.021*"прощать" + 0.021*"чувство" + 0.019*"сделать" + 0.018*"сюда" + '
  '0.018*"ждать" + 0.016*"нина"'),
 (1,
  '0.062*"хотеть" + 0.062*"сказать" + 0.044*"видеть" + 0.039*"который" + '
  '0.035*"маменька" + 0.033*"свой" + 0.028*"деньга" + 0.024*"день" + '
  '0.023*"понимать" + 0.022*"стать"'),
 (2,
  '0.040*"твой" + 0.024*"матушка" + 0.020*"пора" + 0.018*"рука" + 0.017*"друг" '
  '+ 0.016*"сметь" + 0.016*"искусство" + 0.015*"работа" + 0.014*"молодость" + '
  '0.014*"мысль"'),
 (3,
  '0.075*"быть" + 0.037*"господин" + 0.035*"бог" + 0.033*"саша" + '
  '0.032*"горский" + 0.029*"сумасшедший" + 0.028*"этот" + 0.023*"красть" + '
  '0.014*"нищий" + 0.012*"справедливо"'),
 (4,
  '0.048*"ваш" + 0.040*"идти" + 0.035*"брать" + 0.031*"наш" + 0.022*"глаз" + '
  '0.022*"свет" + 0.021*"казаться" + 0.019*"хороший" + 0.019*"угодный" + '
  '0.017*"сидеть"'),
 (5,
  '0.072*"любить" + 0.052*"мой" + 0.039*"жизнь" + 0.037*"год" +

In [ ]:
# Compute Perplexity
Perplexity = lda_model.log_perplexity(corpus)   # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=children_19, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [110]:
print('\nPerplexity: ', Perplexity )
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.260231994944363

Coherence Score:  0.4789933319835115


In [111]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.087375 -0.265478       1        1  12.833087
1      0.049811  0.014026       2        1  12.434628
9      0.014423 -0.011163       3        1  11.714166
5     -0.330797 -0.003064       4        1  11.445632
4      0.018817  0.008284       5        1  10.028040
2      0.089185  0.236713       6        1   9.752934
0      0.016800  0.006811       7        1   9.209008
6      0.015960  0.003230       8        1   8.617029
7      0.020724  0.007674       9        1   7.459407
3      0.017702  0.002968      10        1   6.506069, topic_info=          Term        Freq       Total Category  logprob  loglift
26       знать  300.000000  300.000000  Default  30.0000  30.0000
0     говорить  272.000000  272.000000  Default  29.0000  29.0000
124     любить  188.000000  188.000000  Default  28.0000  28.0000
3       хотеть  177.000000  177.000000  Default  27.0000  27.0000
107    сказать  176.000000  176.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
2638   акулина    8.495701    9.285985  Topic10  -5.1700   2.6435
1719      алеш    9.215974   10.078318  Topic10  -5.0886   2.6430
1473     гроза    8.342089    9.168006  Topic10  -5.1882   2.6380
465    ставить    7.983817    8.776036  Topic10  -5.2321   2.6378
4908   андрюша   13.036589   18.184694  Topic10  -4.7418   2.3996

[345 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
5124      2  0.054216     аккуратно
5124      9  0.759019     аккуратно
5124     10  0.054216     аккуратно
2638     10  0.861513       акулина
313       5  0.969095     александр
...     ...       ...           ...
530       7  0.980607       чувство
1501      3  0.953640        чудной
1382      3  0.961372         чужой
5132      8  0.937014  шестнадцатый
284      10  0.992239          этот

[343 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 10, 6, 5, 3, 1, 7, 8, 4])

## 20 centure

In [ ]:
corpus, id2word = topic_modeling(children_20)

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [114]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.059*"жизнь" + 0.036*"мама" + 0.035*"старик" + 0.032*"человек" + '
  '0.030*"маша" + 0.023*"очень" + 0.022*"таков" + 0.020*"идти" + 0.017*"время" '
  '+ 0.016*"смотреть"'),
 (1,
  '0.099*"знать" + 0.018*"дом" + 0.016*"заявить" + 0.015*"прислать" + '
  '0.013*"обидеться" + 0.013*"скандал" + 0.013*"поцелуй" + 0.013*"любовь" + '
  '0.012*"влюбиться" + 0.012*"имя"'),
 (2,
  '0.071*"сказать" + 0.057*"какой" + 0.036*"почему" + 0.031*"один" + '
  '0.028*"год" + 0.024*"виноватый" + 0.018*"молчать" + 0.015*"неправда" + '
  '0.015*"уйти" + 0.015*"стараться"'),
 (3,
  '0.066*"видеть" + 0.030*"стать" + 0.029*"иванович" + 0.026*"думать" + '
  '0.021*"нужный" + 0.019*"слышать" + 0.016*"сергеевич" + 0.014*"штука" + '
  '0.014*"сложный" + 0.013*"павел"'),
 (4,
  '0.026*"ждать" + 0.025*"княжна" + 0.019*"рука" + 0.018*"эллен" + '
  '0.017*"прелестный" + 0.017*"чтонибыть" + 0.015*"подумать" + 0.015*"её" + '
  '0.015*"первый" + 0.015*"сократ"'),
 (5,
  '0.052*"любить" + 0.035*"казаться" + 0.034*"

In [ ]:
# Compute Perplexity
Perplexity = lda_model.log_perplexity(corpus)   # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=children_20, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [116]:
print('\nPerplexity: ', Perplexity )
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.325303256915332

Coherence Score:  0.5599544837551791


In [117]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.011040 -0.311043       1        1  12.060056
6      0.297345  0.061086       2        1  11.358987
8      0.013332  0.043839       3        1  10.501672
9     -0.137145  0.044161       4        1  10.268562
7     -0.039096  0.020586       5        1  10.118758
1     -0.088060  0.084339       6        1   9.888353
4     -0.027279  0.020988       7        1   9.499076
5     -0.018536  0.017410       8        1   8.901197
3     -0.008478  0.013522       9        1   8.890603
2     -0.003122  0.005113      10        1   8.512736, topic_info=         Term       Freq      Total Category  logprob  loglift
308     знать  52.000000  52.000000  Default  30.0000  30.0000
137   сказать  32.000000  32.000000  Default  29.0000  29.0000
152     жизнь  38.000000  38.000000  Default  28.0000  28.0000
196    видеть  31.000000  31.000000  Default  27.0000  27.0000
200      дать  33.000000  33.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
1561     федя   2.387889   2.997564  Topic10  -5.2445   2.2362
1533   хозяин   2.368762   2.978470  Topic10  -5.2525   2.2346
478    тысяча   2.237286   2.846811  Topic10  -5.3096   2.2227
1726    плата   2.007777   2.617527  Topic10  -5.4178   2.1984
972   сколько   2.879915   5.728592  Topic10  -5.0571   1.7759

[347 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1967      4  0.970462      андрей
1501      2  0.349940        анна
1501      3  0.612394        анна
1477      7  0.964152       анюта
2053      2  0.960781  беспощадно
...     ...       ...         ...
1522      5  0.803214        чёрт
542       3  0.837735      шалить
2073      9  0.946998       штука
1618      7  0.945911       эллен
1887      8  0.905913   ювелирный

[324 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 9, 10, 8, 2, 5, 6, 4, 3])